In [28]:
import json
import tensorflow as tf
import numpy as np
from typing import Optional
from src.models.ig_cnn import ig_cnn_model


In [7]:
def gen_conv(filters:int, kernel_size:int, activation:Optional[str]=None, padding:Optional[str]=None):
   _conv = f"C-{filters}-{kernel_size}"
   if activation is not None:
       _conv += f"-{activation}"
   if padding is not None:
       _conv += f"-{padding}"
   return _conv

def gen_dense(units: int, activation: Optional[str]=None):
    _dense = f"D-{units}"
    if  activation is not None:
       _dense += f"-{activation}"
    return _dense

### Generate CNN configs

In [8]:
idx = 0
for phantom_size in [3, 5]: # 7]:
    print(f"Input {phantom_size}")
    for activation in ['sigmoid', 'relu']:
        for filters in [10, 15, 25, 50]:
            for kernel in [3, 5, 7]:
                i = phantom_size
                depth = 0
                layers = ""
                while i >= kernel:
                    depth += 1
                    i -= (kernel-np.floor(kernel/2.0))
                    layers += f"{gen_conv(filters, kernel, activation, padding='valid')},"
                    # Test if model is creatable
                    model_def = f"{layers}F"
                    model = ig_cnn_model(model_def, input_size=(phantom_size, phantom_size))
                    model.compile(loss='mse')
                    with open(f"{idx}.json", 'w') as file:
                        config = dict(model_def=model_def,
                                      activation=activation,
                                      kernel=kernel,
                                      filters=filters,
                                      depth=depth,
                                      phantom_size=phantom_size,
                                      log_name=f'{phantom_size}/{model_def}/',
                                      log_root='../logs/',
                                      phantom_root=f'../data/phantoms/phantom_{phantom_size}_{phantom_size}/training',
                                      phantom_file_wildcard=f'*_phantom_{phantom_size}_{phantom_size}.npz'
                                 )
                        json.dump(config, file)
                    idx += 1

Input 3
Input 5


### Generate Baseline configs

In [31]:
idx = 0
for phantom_size in [3, 5]:
    # Test if model is creatable
    model_def = f'dense_baseline_{phantom_size}'
    with open(f"baseline_{phantom_size}.json", 'w') as file:
        config = dict(model_def=model_def,
                      phantom_size=phantom_size,
                      log_name=f'{phantom_size}/baseline/',
                      log_root='../logs/',
                      phantom_root=f'../data/phantoms/phantom_{phantom_size}_{phantom_size}/training',
                      phantom_file_wildcard=f'*_phantom_{phantom_size}_{phantom_size}.npz'
                 )
        json.dump(config, file)
    idx += 1

Results

In [ ]:
import pandas as pd

In [22]:
df = pd.read_csv('hparams_table.csv')
df.groupby(['model_def', 'phantom_size'])

In [26]:
df[['model_def', 'phantom_size', 'Information Gain valid', 'Information Gain valid.1', 'epoch_loss.1']].rename(columns={
     'Information Gain valid': 'ig_max_valid',
     'Information Gain valid.1': 'ig_mean_valid',
     'epoch_loss.1': 'mse_valid'
}).groupby(['model_def', 'phantom_size']).agg(['mean', 'std', 'median'])

ig_max_valid  \
                                                                 mean   
model_def                                   phantom_size                
C-10-3-relu-valid,C-10-3-relu-valid,F       5.0              0.299669   
C-10-3-relu-valid,F                         3.0              0.324549   
                                            5.0              0.310608   
C-10-3-sigmoid-valid,C-10-3-sigmoid-valid,F 5.0              0.284301   
C-10-3-sigmoid-valid,F                      3.0              0.291812   
                                            5.0              0.313462   
C-10-5-relu-valid,F                         5.0              0.304839   
C-10-5-sigmoid-valid,F                      5.0              0.309827   
C-15-3-relu-valid,C-15-3-relu-valid,F       5.0              0.276457   
C-15-3-relu-valid,F                         3.0              0.288449   
                                            5.0              0.297855   
C-15-3-sigmoid-valid,C-15-3-sigmoid-valid,F 5.0              0.275681   
C-15-3-sigmoid-valid,F                      3.0              0.287962   
                                            5.0              0.305602   
C-15-5-relu-valid,F                         5.0              0.301777   
C-15-5-sigmoid-valid,F                      5.0              0.302920   
C-25-3-relu-valid,C-25-3-relu-valid,F       5.0              0.251965   
C-25-3-relu-valid,F                         3.0              0.282944   
                                            5.0              0.287415   
C-25-3-sigmoid-valid,C-25-3-sigmoid-valid,F 5.0              0.272152   
C-25-3-sigmoid-valid,F                      3.0              0.296784   
                                            5.0              0.302478   
C-25-5-relu-valid,F                         5.0              0.302182   
C-25-5-sigmoid-valid,F                      5.0              0.303393   
C-50-3-relu-valid,C-50-3-relu-valid,F       5.0              0.276533   
C-50-3-relu-valid,F                         3.0              0.270527   
                                            5.0              0.282089   
C-50-3-sigmoid-valid,C-50-3-sigmoid-valid,F 5.0              0.271588   
C-50-3-sigmoid-valid,F                      3.0              0.293388   
                                            5.0              0.300167   
C-50-5-relu-valid,F                         5.0              0.300778   
C-50-5-sigmoid-valid,F                      5.0              0.296331   

                                                                              \
                                                               std    median   
model_def                                   phantom_size                       
C-10-3-relu-valid,C-10-3-relu-valid,F       5.0           0.087406  0.320398   
C-10-3-relu-valid,F                         3.0           0.084814  0.323633   
                                            5.0           0.085954  0.321780   
C-10-3-sigmoid-valid,C-10-3-sigmoid-valid,F 5.0           0.072453  0.300132   
C-10-3-sigmoid-valid,F                      3.0           0.081829  0.312518   
                                            5.0           0.092333  0.332465   
C-10-5-relu-valid,F                         5.0           0.074567  0.329762   
C-10-5-sigmoid-valid,F                      5.0           0.084064  0.311352   
C-15-3-relu-valid,C-15-3-relu-valid,F       5.0           0.068310  0.284423   
C-15-3-relu-valid,F                         3.0           0.075780  0.290756   
                                            5.0           0.081476  0.295248   
C-15-3-sigmoid-valid,C-15-3-sigmoid-valid,F 5.0           0.070468  0.270719   
C-15-3-sigmoid-valid,F                      3.0           0.085514  0.304237   
                                            5.0           0.095199  0.303558   
C-15-5-relu-valid,F                         5.0           0.082657  0.314247   
C-15-5-sigmoid-valid,F                      5.0           0.0